In [1]:
import LinearAlgebra: I, ⋅
import Base.MathConstants: φ, pi
import Base: cos ,Iterators
import Statistics: mean
abstract type DescentMethod end

In [2]:
using PyPlot
using BenchmarkTools
using Zygote

# Metody optymalizacji

## Metoda najszybszego spadku z bezwładnością

In [15]:
mutable struct Momentum <: DescentMethod
  α#::Array{Float64} # learning rate
  β#::Array{Float64} # momentum decay
  v#::Float64# momentum
end

Momentum(α, β, n::Integer) = Momentum(α, β, zeros(n))

function step!(M::Momentum, f, ∇f, x::Array{Float64}, debug=false) 
  α, β, v, g = M.α, M.β, M.v, ∇f(x)
  if debug
    println("Gradient: ", g)
    println(M)
  end
  v[:]  = β*v .- α*g
  return x + v
end

step! (generic function with 4 methods)

## BFGS

In [4]:
mutable struct BFGS <: DescentMethod
  Q
end
BFGS(n::Integer) = BFGS(Matrix(1.0I, n, n))

function strong_backtracking(f, ∇, x, d; α=1, β=1e-4, σ=0.1)
  y0, g0, y_prev, α_prev = f(x), ∇(x)⋅d, NaN, 0
  αlo, αhi = NaN, NaN
  # bracket phase
  while true
    y = f(x + α*d)
    if y > y0 + β*α*g0 || (!isnan(y_prev) && y ≥ y_prev)
      αlo, αhi = α_prev, α
      break
    end
    g = ∇(x + α*d)⋅d
    if abs(g) ≤ -σ*g0
      return α
    elseif g ≥ 0
      αlo, αhi = α, α_prev
      break
    end
    y_prev, α_prev, α = y, α, 2α
  end
  # zoom phase
  ylo = f(x + αlo*d)
  while true
    α = (αlo + αhi)/2
    y = f(x + α*d)
    if y > y0 + β*α*g0 || y ≥ ylo
      αhi = α
    else
      g = ∇(x + α*d)⋅d
      if abs(g) ≤ -σ*g0
        return α
      elseif g*(αhi - αlo) ≥ 0
        αhi = αlo
      end
      αlo = α
    end
  end
end

function step!(M::BFGS, f, ∇f, x)
  if f(x) ≈ 0.0
    return x
  end

  Q, g = M.Q, ∇f(x)
  α = strong_backtracking(f, ∇f, x, -Q*g)
  x′ = x + α*(-Q*g)
  g′ = ∇f(x′)
  δ = x′ - x
  γ = g′ - g
  Q[:] = Q - (δ*γ'*Q + Q*γ*δ')/(δ'*γ) + (1 + (γ'*Q*γ)/(δ'*γ))[1]*(δ*δ')/(δ'*γ)
  return x′
end

step! (generic function with 3 methods)

## L-BFGS

In [60]:
using LinearAlgebra

mutable struct LBFGS
  m::Float64
  δs#::Array{Float64}
  γs#::Array{Float64}
  qs#::Array{Float64}
  LBFGS() = new()
end

function init!(M::LBFGS, m) 
  M.m = m
  M.δs = [] 
  M.γs = [] 
  M.qs = []
  return M
end

function step!(M::LBFGS, f, ∇f, θ) 
    δs, γs, qs = M.δs, M.γs, M.qs 
    m, g = length(δs), ∇f(θ)
    d = -g # kierunek
    # if isnan(g)
        # there is no dericative at θ
        # we can't progress any further
        # posibli move in random direction
        # return θ || (!isnan(y_prev) && y ≥ y_prev)
    if m > 0 
        q = g
        for i in m:-1:1
            qs[i] = copy(q)
            q -= (δs[i]⋅q) / (γs[i]⋅δs[i]) * γs[i]
        end
        z = (γs[m] .* δs[m] .* q) / (γs[m]⋅γs[m]) 
        for i in 1:+1:m
            z += δs[i]*(δs[i]⋅qs[i]-γs[i]⋅z)/(γs[i]⋅δs[i]) 
        end
        d = -z; # rekonstrukcja kierunku
    end
    φ =α-> f(θ+α*d); φ′=α->∇f(θ+α*d)⋅d 
    α = line_search(φ, φ′, d)
    θ′ = θ + α*d; g′ = ∇f(θ′) # nowy wektor
    δ =θ′-θ;γ =g′-g
    push!(δs, δ);
    push!(γs, γ);
    push!(qs, zero(θ)) 
    while length(δs) > M.m
        popfirst!(δs); popfirst!(γs); popfirst!(qs) 
    end
    return θ′ 
end

step! (generic function with 4 methods)

In [6]:
function zoom(φ, φ′, αlo, αhi, c1=1e-4, c2=0.1, jmax=1000)
    φ′0 = φ′(0.0) 
    for j=1:jmax
        αj = 0.5(αlo + αhi) # bisection 
        φαj = φ(αj)
        if φαj > φ(0.0) + c1*αj*φ′0 || φαj ≥ φ(αlo)
            αhi = αj 
        else
            φ′αj = φ′(αj)
            if abs(φ′αj) ≤ -c2*φ′0
                return αj
            end
            if φ′αj*(αhi - αlo) ≥ 0.0 
                αhi = αlo
            end
            αlo = αj 
        end
    end
    return 0.5(αlo + αhi) 
end

function line_search(φ, φ′, d, c1=1e-4, c2=0.1, ρ=0.1, αmax=100., jmax=1000)
    αi, αj = 0.0, 1.0
    φαi, φ0, φ′0 = φ(αi), φ(0.0), φ′(0.0) 
    for j=1:jmax
        φαj = φ(αj)
        if φαj > φ0 + c1*αj*φ′0 || (φαj ≥ φαi && j > 1)
            return zoom(φ, φ′, αi, αj)
        end
        φ′αj = φ′(αj)
        if abs(φ′αj) ≤ -c2*φ′0
            return αj 
        end
        if φ′αj ≥ 0.0
            return zoom(φ, φ′, αj, αi)
        end
        αi, αj = αj, ρ*αj + (1.0 - ρ)*αmax
        φαi = φαj 
    end
    return αj 
end

line_search (generic function with 6 methods)

# Test Set up

## Functions

In [7]:
f_beale(x) = (1.5 - x[1] + x[1]*x[2])^2 + (2.25 - x[1] + x[1]*x[2]^2)^2 + (2.625 - x[1] + x[1]*x[2]^3)^2
∇f_beale(x) = [
    2*x[1]*(x[2]^6 + x[2]^4 - 2*x[2]^3 - x[2]^2 - 2*x[2] + 3) + 5.25*x[2]^3 + 4.5*x[2]^2 + 3*x[2] - 12.75, 
    6*x[1]*(x[1]*(x[2]^5 + 0.666667*x[2]^3 - x[2]^2 - 0.333333*x[2] - 0.333333) + 2.625*x[2]^2 + 1.5x[2] + 0.5)
]

f_rosenbrock(x)  = 100*(x[2] - x[1]^2)^2 + (1-x[1])^2
∇f_rosenbrock(x) = [
    400x[1]^3 - 400x[1]*x[2] + 2x[1] - 2,
    200x[2] - 200x[1]^2
]
# a = 20, b = 0.2 and c = 2π.
f_ackley(x, a=20,b=0.2,c = 2 * pi) = -a * exp(-b * sqrt(mean(x.^2))) - exp( mean( cos.(c .* x))) + a + exp(1)
∇f_ackley(x) = f_ackley'(x)

x = [3.0, 2.4]

2-element Array{Float64,1}:
 3.0
 2.4

# Test

In [19]:
function optimalize(f, ∇f, x₀, opt, e, i, debug=false)
    pts = [x₀] # kolejne wektory x
    err = Float64[] # kolejne wartości f. straty
    p = 0
    while true
        prev = f(pts[end])
        push!(err, prev) # odłóż wynik funkcji dla najnowszego wektora x (miara błędu)
        if debug 
            println("Iteracja p=", p)
            println("Wektor x: ", pts[end])
            println("Error: ", err[end])
            println()
        end
        if prev < e || isnan(prev)  || p > i 
            break
        end
        # if length(pts) > 1 && pts[end] == pts[end-1]
        # break # when we stop to progress
        push!(pts, step!(opt, f, ∇f, pts[end]))
        p += 1
    end
    
    pts, err, p
end

optimalize (generic function with 2 methods)

## ty


In [17]:
bfgs = BFGS(2)

pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, bfgs, 0.0001, 100, false)
println("Liczba iteracji: ", i)
println("Wektor x wynikowy: ", pts[end])
println("Błąd: ", err[end])
# println(info)

Liczba iteracji: 20
Wektor x wynikowy: [0.9933245206924979, 0.9864200092686151]
Błąd: 5.2047399349124887e-5


In [70]:
lbfgs = LBFGS(); init!(lbfgs, 7)

pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, lbfgs, 0.0001, 100)
println("Liczba iteracji: ", i)
println("Wektor x wynikowy: ", pts[end])
println("Błąd: ", err[end])

Liczba iteracji: 7
Wektor x wynikowy: [NaN, NaN]
Błąd: NaN


In [39]:
# weird stuff happens here
x = [3.0,4.0]
f = f_rosenbrock
∇f = ∇f_rosenbrock
momentum = Momentum(0.00000000000001, 0.01, 2)

pts, err, i = optimalize(f, ∇f, x, momentum, 0.01, 100)

println("Liczba iteracji: ", i)
println("Wektor x wynikowy: ", pts[end])
println("Błąd: ", err[end])

Momentum(1.0e-14, 0.01, [0.0, 0.0])
Liczba iteracji: 101
Wektor x wynikowy: [2.9999999938753064, 4.000000001020124]
Błąd: 2503.999962207215


## Ackley test

### setup

In [44]:

x = fill(15.0,4)
f = f_ackley
∇f = ∇f_ackley

∇f_ackley (generic function with 1 method)

In [69]:
∇f(x)
f(x)

810121.0

In [68]:
# weird stuff happens here
momentum = Momentum(0.00000000000001, 0.01, length(x))
pts, err, i = optimalize(f, ∇f, x, momentum, 0.01, 100)
println("Liczba iteracji: ", i)
println("Wektor x wynikowy: ", pts[end])
println("Błąd: ", err[end])

Liczba iteracji: 101
Wektor x wynikowy: [-9.999999632741533, 10.00000001836188]
Błąd: 810120.8674483662


### BFGS

In [47]:

lbfgs = LBFGS(); init!(lbfgs, 1)
pts, err, i = optimalize(f, ∇f, x, lbfgs, 0.0001, 100,true)
println("Liczba iteracji: ", i)
println("Wektor x wynikowy: ", pts[end])
println("Błąd: ", err[end])

Iteracja p=0
Wektor x: [15.0, 15.0, 15.0, 15.0]
Error: 19.00425863264272

[0.049787068367817926, 0.049787068367817926, 0.049787068367817926, 0.049787068367817926]Iteracja p=1
Wektor x: [14.998055192641882, 14.998055192641882, 14.998055192641882, 14.998055192641882]
Error: 19.004074186734965

[-0.0023643732097136175, -0.0023643732097136175, -0.0023643732097136175, -0.0023643732097136175]Iteracja p=2
Wektor x: [14.998143363750417, 14.998143363750417, 14.998143363750417, 14.998143363750417]
Error: 19.004073769716804

[-4.395188207689804e-7, -4.395188207689804e-7, -4.395188207689804e-7, -4.395188207689804e-7]Iteracja p=3
Wektor x: [14.998143380143796, 14.998143380143796, 14.998143380143796, 14.998143380143796]
Error: 19.00407376971679

[5.750448728303326e-12, 5.750448728303326e-12, 5.750448728303326e-12, 5.750448728303326e-12]Iteracja p=4
Wektor x: [14.998143380143581, 14.998143380143581, 14.998143380143581, 14.998143380143581]
Error: 19.00407376971679

[-1.0297318553398327e-14, -1.0297318

## Test Wydajności

In [53]:
x = [-10.0, 10.0]
f = f_rosenbrock
∇f = ∇f_rosenbrock

∇f_rosenbrock (generic function with 1 method)

In [57]:

momentum = Momentum(0.00000000000001, 0.01, length(x))
@benchmark pts, err, i = optimalize(f, ∇f, x, momentum, 0.01, 100)

BenchmarkTools.Trial: 
  memory estimate:  58.16 KiB
  allocs estimate:  724
  --------------
  minimum time:     88.700 μs (0.00% GC)
  median time:      104.000 μs (0.00% GC)
  mean time:        158.807 μs (7.99% GC)
  maximum time:     12.998 ms (98.76% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [58]:
bfgs = BFGS(length(x))

@benchmark pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, bfgs, 0.0001, 100, false)

BenchmarkTools.Trial: 
  memory estimate:  17.34 KiB
  allocs estimate:  225
  --------------
  minimum time:     21.499 μs (0.00% GC)
  median time:      158.750 μs (0.00% GC)
  mean time:        329.059 μs (10.31% GC)
  maximum time:     71.063 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [67]:
lbfgs = LBFGS(); init!(lbfgs, 1)
@benchmark pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, lbfgs, 0.0001, 100)

BenchmarkTools.Trial: 
  memory estimate:  628.20 KiB
  allocs estimate:  15060
  --------------
  minimum time:     607.500 μs (0.00% GC)
  median time:      731.700 μs (0.00% GC)
  mean time:        1.471 ms (9.27% GC)
  maximum time:     17.397 ms (95.02% GC)
  --------------
  samples:          3345
  evals/sample:     1

In [65]:
lbfgs = LBFGS(); init!(lbfgs, 2)

@benchmark pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, lbfgs, 0.0001, 100)

BenchmarkTools.Trial: 
  memory estimate:  628.94 KiB
  allocs estimate:  15075
  --------------
  minimum time:     607.300 μs (0.00% GC)
  median time:      697.699 μs (0.00% GC)
  mean time:        1.125 ms (10.97% GC)
  maximum time:     14.616 ms (91.28% GC)
  --------------
  samples:          4341
  evals/sample:     1

In [63]:
lbfgs = LBFGS(); init!(lbfgs, 3)

@benchmark pts, err, i = optimalize(f_rosenbrock, ∇f_rosenbrock, x, lbfgs, 0.0001, 100)

BenchmarkTools.Trial: 
  memory estimate:  629.67 KiB
  allocs estimate:  15090
  --------------
  minimum time:     641.300 μs (0.00% GC)
  median time:      949.149 μs (0.00% GC)
  mean time:        2.135 ms (7.94% GC)
  maximum time:     34.703 ms (84.43% GC)
  --------------
  samples:          2282
  evals/sample:     1